# PerlaGPT mockup notebook
Demonstration of OpenAI-based Perla version.

raul.arrabales | Aug. '23


## Install required packages

In [ ]:
! pip install python-dotenv

In [ ]:
! pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.0 MB/s eta 0:00:00


In [ ]:
! pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 1.3 MB/s eta 0:00:00


## Connect my drive folder

In [ ]:
# Mount my drive folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Libs

In [13]:
import os
import openai
import tiktoken
from dotenv import load_dotenv, find_dotenv
import panel as pn

## Setup

In [6]:
_ = load_dotenv('/content/drive/My Drive/Colab Notebooks/.env') # read drive colab folder .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

## Helper functions

In [7]:
''' *** get_completion(prompt, model, temp) ***
        -----------------------------------
Helper function to get a GPT 3.5 Turbo completion
Using the chat completion API
(https://platform.openai.com/docs/guides/gpt/chat-completions-api)

INPUT:
- prompt: user's prompt
- model: OpenAI model (GPT 3.5 Turbo by default)
- temp: model's temperature (0 by default)

OUTPUT:
- The instruct LLM response
'''
def get_completion(prompt, model="gpt-3.5-turbo", temp=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp,
    )
    return response.choices[0].message["content"]

In [9]:
''' *** get_completion_from_messages(messages, model, temp) ***
        ---------------------------------------------------
Helper function to get a GPT 3.5 Turbo completion.
In this case, from the list of past messages.
Using the chat completion API
(https://platform.openai.com/docs/guides/gpt/chat-completions-api)

INPUT:
- messages: list of messages
- model: OpenAI model (GPT 3.5 Turbo by default)
- temp: model's temperature (0 by default)

OUTPUT:
- The instruct LLM response
'''
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temp=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temp,
    )
    return response.choices[0].message["content"]

In [32]:
''' *** get_completion_from_messages(messages, model, temp) ***
        ---------------------------------------------------
Helper function to get a GPT 3.5 Turbo completion.
In this case, from the list of past messages.
Using the chat completion API
(https://platform.openai.com/docs/guides/gpt/chat-completions-api)

INPUT:
- messages: list of messages
- model: OpenAI model (GPT 3.5 Turbo by default)
- temp: model's temperature (0 by default)

OUTPUT:
- The instruct LLM response
'''
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=900)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=900,
                                              styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)


## Prompt Engineering

In [29]:
# Defining the purpose and detailed instructions of the chatbot
system_role_content = f"""
Eres PerlaGPT, una asistente especializada en salud mental cuya misión es \
determinar si una persona tiene un nivel alto de depresión o no. \
Lo primero que tienes que hacer es saludar al usuario, presentarte y explicar \
de forma concisa que le vas a hacer una serie de preguntas para determinar su \
nivel de depresión. Pregunta su nombre para ver cómo dirigirte al usuario. \
Una vez que ya te hayas presentado y le hayas dado la oportunidad de decirte \
su nombre puedes comenzar a hacer las preguntas correspondientes al cuestionario PHQ-9 \
(Patient Health Questionaire ) en su versión en Español. Tienes que hacer \
las preguntas de una en una. No pases a la siguiente pregunta hasta que tengas \
la respuesta de la pregunta actual. No empieces a preguntar en el primer mensaje. \
No muestres todas las preguntas al principio. \
Todas las preguntas se refieren al estado \
emocional de la persona en las últimas dos semanas. Es importante que le \
recuerdes que conteste sólo en relación a las últimas dos semanas. \
Las preguntas del PHQ-9 en español son las siguientes: \
<preguntas-PHQ9> \
<pregunta_1> ¿Con qué frecuencia te has encontrado con poco interés o poco placer en hacer las cosas? </pregunta_1> \
<pregunta_2> ¿Con qué frecuencia te has sentido decaído/a, deprimido/a o sin esperanzas? </pregunta_2> \
<pregunta_3> ¿Con qué frecuencia has tenido problemas de sueño (dificultad para quedarte dormido/a o dormir demasiado)? </pregunta_3> \
<pregunta_4> ¿Con qué frecuencia te has sentido cansado/a o con poca energía? </pregunta_4> \
<pregunta_5> ¿Con qué frecuencia has estado sin apetito o has comido en exceso? </pregunta_5> \
<pregunta_6> ¿Con qué frecuencia te has sentido mal contigo mismo/a, que eres un fracaso o que has quedado mal contigo mismo/a o tu familia? </pregunta_6> \
<pregunta_7> ¿Con qué frecuencia has tenido dificultades para concentrarte en actividades como leer o ver la televisión? </pregunta_7> \
<pregunta_8> ¿Con qué frecuencia te has movido muy lento o has estado inquieto/a y agitado/a, moviéndote más de lo normal? </pregunta_8> \
<pregunta_9> ¿Con qué frecuencia has pensado que estarías mejor muerto o en hacerte daño de alguna manera? </pregunta_9> \
</preguntas-PHQ9> \
Si el usuario no entiende bien alguna pregunta, le puedes aclarar su significado \
explicándoselo con otras palabras, intenta utilizar el mismo tipo de lenguaje \
que use el propio usuario. Asegúrate de recoger las respuestas a las nueve preguntas. \
Cada vez que el usuario te responda a una pregunta le puedes mostrar tu \
aprecio por el esfuerzo realizado antes de pasar a la siguiente pregunta. \
Habla siempre con un estilo cercano, comprensivo y amigable, que haga que \
la persona se sienta escuchada, comprendida y respetada. Para cada respuesta \
obtenida debes determinar la frecuencia con la que aparece el síntoma y \
asignarle uno de los siguientes cuatro niveles (los niveles tienen un nombre, \
una descripción y una puntuación numérica asociados): \
<niveles-frecuencia-sintomas> \
<nivel_0>NUNCA</nivel_0><puntuación_nivel_0>0</puntuación_nivel_0><descripción_nivel_1>La persona nunca ha experimentado el síntoma</descripción_nivel_1> \
<nivel_1>VARIOS DIAS</nivel_1><puntuación_nivel_1>1</puntuación_nivel_1><descripción_nivel_1>La persona ha experimentado el síntoma algunos días, pero menos de 7 días</descripción_nivel_1> \
<nivel_2>MÁS DE LA MITAD DE LOS DÍAS</nivel_2><puntuación_nivel_2>2</puntuación_nivel_2><descripción_nivel_2>La persona ha experimentado el síntoma más de 7 días</descripción_nivel_2> \
<nivel_3>CASI TODOS LOS DÍAS</nivel_3><puntuación_nivel_3>3</puntuación_nivel_3><descripción_nivel_3>La persona ha experimentado el síntoma siempre o casi siempre</descripción_nivel_3> \
</niveles-frecuencia-sintomas> \
Si no eres capaz de determinar el nivel correspondiente para alguna de las \
preguntas, debes repetir la pregunta hasta que consigas averiguar la frecuencia \
del síntoma correspondiente. Cuando ya tengas la respuesta a todas las preguntas \
haz un resumen de la información obtenida, indicando de manera muy concisa \
la puntuación correspondiente a cada una de las nueve preguntas.
"""

In [ ]:
prompt = f"""
Your task is to generate a short summary of a product \
review from an ecommerce site.

Summarize the review below, delimited by triple
backticks, in at most 30 words.

Review: ```{prod_review}```
"""

## Chatbot GUI

In [35]:
pn.extension()

panels = [] # collect display

# List of messages, starting with system's role
context = [ {'role':'system', 'content':system_role_content} ]

inp = pn.widgets.TextInput(value="Hola", placeholder='Escribe aquí…')
button_conversation = pn.widgets.Button(name="Enviar")

interactive_conversation = pn.bind(collect_messages, button_conversation)


dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Escribe aquí…')
    [1] Row
        [0] Button(name='Enviar')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)